In [1]:
import torch
import torch.nn as nn


In [2]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [3]:
class skipconnections(nn.Module):
    def __init__ (self,isskip,layersize):
        super().__init__()
        self.isskip=isskip
        self.layers=nn.ModuleList([
            nn.Sequential(nn.Linear(layersize[0],layersize[1]),GELU()),
            nn.Sequential(nn.Linear(layersize[1],layersize[2]),GELU()),
            nn.Sequential(nn.Linear(layersize[2],layersize[3]),GELU()),
            nn.Sequential(nn.Linear(layersize[3],layersize[4]),GELU()),
            nn.Sequential(nn.Linear(layersize[4],layersize[5]),GELU()),
        ])
    
    def forward(self,x):
        for layer in self.layers:
            layer_output=layer(x)

            if self.isskip and layer_output.shape==x.shape:
                x=x+layer_output
            else:
                x=layer_output
    
        return x

In [7]:
def print_gradients(model, x):
    output = model(x)
    target = torch.tensor([[0.]])


    loss = nn.MSELoss()
    loss = loss(output, target)
    
    loss.backward()

    for name, param in model.named_parameters():
        if 'weight' in name:
            print(f"{name} has gradient mean of {param.grad.abs().mean().item()}")


layer_sizes = [3, 3, 3, 3, 3, 1]  

sample_input = torch.tensor([[1., 0., -1.]])

torch.manual_seed(123)
model_without_shortcut = skipconnections(
    False,layer_sizes
)
print_gradients(model_without_shortcut, sample_input)
print()

torch.manual_seed(123)
model_with_shortcut = skipconnections(
    True,layer_sizes
)
print_gradients(model_with_shortcut, sample_input)

layers.0.0.weight has gradient mean of 0.00020173587836325169
layers.1.0.weight has gradient mean of 0.0001201116101583466
layers.2.0.weight has gradient mean of 0.0007152041071094573
layers.3.0.weight has gradient mean of 0.0013988735154271126
layers.4.0.weight has gradient mean of 0.005049645435065031

layers.0.0.weight has gradient mean of 0.22169791162014008
layers.1.0.weight has gradient mean of 0.20694106817245483
layers.2.0.weight has gradient mean of 0.32896995544433594
layers.3.0.weight has gradient mean of 0.2665732204914093
layers.4.0.weight has gradient mean of 1.3258540630340576
